# Cognee를 사용하여 지속적인 메모리를 가진 AI 에이전트 구축하기

이 노트북은 [**cognee**](https://www.cognee.ai/)를 사용하여 지식 그래프, 의미 검색, 세션 관리를 결합해 컨텍스트를 인식하는 AI 시스템을 만드는 방법을 보여줍니다. Cognee는 오픈 소스 AI 메모리입니다.

## 🎯 학습 목표

이 튜토리얼을 마치면 다음을 이해할 수 있습니다:
- **임베딩 기반 지식 그래프 구축**: 비구조화된 텍스트를 구조화되고 쿼리가 가능한 지식으로 변환하기
- **세션 메모리 구현**: 자동으로 컨텍스트를 유지하는 다중 턴 대화 생성하기
- **대화 저장**: 중요한 상호작용을 장기 메모리에 선택적으로 저장하여 나중에 참조하기
- **자연어로 쿼리하기**: 새로운 대화에서 과거 컨텍스트를 접근하고 활용하기
- **메모리 시각화**: 에이전트의 지식 그래프에서 관계 탐색하기


## 🏗️ 무엇을 만들게 될까요

이 튜토리얼에서는 지속적인 메모리를 가진 **코딩 어시스턴트**를 만들어볼 것입니다. 이 어시스턴트는 다음과 같은 기능을 갖추고 있습니다:

### 1. **지식 베이스 구축**
   - 개발자의 프로필과 전문성 정보를 수집
   - Python 프로그래밍 원칙과 모범 사례를 처리
   - 개발자와 AI 어시스턴트 간의 대화 기록 저장

### 2. **세션 인식 대화**
   - 동일한 세션 내에서 여러 질문 간의 맥락을 유지
   - 각 질문/답변 쌍을 자동으로 캐시하여 효율적으로 검색
   - 대화 기록을 기반으로 일관성 있고 맥락에 맞는 응답 제공

### 3. **장기 메모리**
   - 중요한 대화를 장기 메모리에 저장
   - 지식 베이스와 과거 세션에서 관련 기억을 검색하여 새로운 상호작용에 반영
   - 시간이 지남에 따라 개선되는 성장형 지식 베이스 구축

### 4. **지능형 메모리 검색**
   - 그래프 인식 의미론적 검색을 사용하여 저장된 모든 지식에서 관련 정보를 찾음
   - 데이터 하위 그룹(개발자 정보 vs. 원칙)별로 검색 필터링
   - 여러 데이터 소스를 결합하여 포괄적인 답변 제공


## 📋 사전 준비 및 설정

### 시스템 요구 사항

시작하기 전에 다음을 확인하세요:

1. **Python 환경**
   - Python 3.9 이상
   - 가상 환경 (권장)

2. **Redis 캐시** (세션 관리를 위해 필요)
   - 로컬 Redis: `docker run -d -p 6379:6379 redis`
   - 또는 관리형 Redis 서비스 사용

3. **LLM API 접근**
   - OpenAI API 키 또는 기타 제공자 (자세한 내용은 [문서](https://docs.cognee.ai/setup-configuration/llm-providers) 참조)

4. **데이터베이스 구성**
   - 기본적으로 별도의 설정이 필요하지 않습니다. Cognee는 파일 기반 데이터베이스(LanceDB 및 Kuzu)를 사용합니다.
   - 선택적으로 Azure AI Search를 벡터 스토어로 설정할 수 있습니다 (자세한 내용은 [문서](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch) 참조).

### 환경 구성

프로젝트 디렉토리에 `.env` 파일을 생성하고 다음 변수를 추가하세요:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Cognee의 메모리 아키텍처 이해하기

### Cognee 작동 방식

Cognee는 단순한 키-값 저장소를 넘어서는 정교한 메모리 시스템을 제공합니다:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### 주요 구성 요소:

1. **지식 그래프**: 엔티티, 관계, 의미적 연결을 저장
2. **벡터 임베딩**: 저장된 모든 정보에 대한 의미적 검색 가능
3. **세션 캐시**: 세션 내 및 세션 간 대화 컨텍스트 유지
4. **노드셋(NodeSets)**: 데이터를 논리적 카테고리로 구성하여 목표 지향적 검색 가능

### 이 튜토리얼에서 다룰 메모리 유형:

- **영구 메모리**: 지식 그래프에 장기적으로 저장
- **세션 메모리**: Redis 캐시에 임시 대화 컨텍스트 저장
- **의미적 메모리**: 모든 데이터에 대한 벡터 기반 유사성 검색


## 📦 필수 패키지 설치

세션 관리를 위한 Redis 지원과 함께 Cognee를 설치하세요:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 환경 초기화 및 라이브러리 로드

확인하세요:
1. Redis가 실행 중인지 확인 (예: Docker를 통해 `docker run -d -p 6379:6379 redis` 실행)
2. 환경 변수가 캐시 모듈을 가져오기 전에 설정되어 있는지 확인
3. 필요 시 커널을 재시작하고 셀을 순서대로 실행

다음 셀은 다음 작업을 수행합니다:
1. `.env` 파일에서 환경 변수를 로드
2. Cognee를 LLM 설정으로 구성
3. 세션 관리를 위한 캐싱 활성화
4. 모든 구성 요소가 제대로 연결되었는지 확인


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 저장 디렉토리 구성

Cognee는 작업을 위해 두 개의 별도 디렉토리를 사용합니다:
- **데이터 루트**: 가져온 문서와 처리된 데이터를 저장
- **시스템 루트**: 지식 그래프 데이터베이스와 시스템 메타데이터를 포함

이 튜토리얼에서는 다음과 같이 독립된 디렉토리를 생성하겠습니다:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 메모리 상태 초기화

메모리 시스템을 구축하기 전에, 새롭게 시작할 수 있도록 준비합시다.

> 💡 **팁**: 나중에 이 노트북을 사용할 때 이전 실행에서의 기존 메모리를 유지하고 싶다면 이 단계를 건너뛸 수 있습니다.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 1부: 지식 베이스 구축

### 개발자 도우미를 위한 데이터 소스

포괄적인 지식 베이스를 만들기 위해 세 가지 유형의 데이터를 수집합니다:

1. **개발자 프로필**: 개인의 전문 지식과 기술적 배경
2. **Python 모범 사례**: Python의 철학과 실용적인 가이드라인
3. **과거 대화 기록**: 개발자와 AI 도우미 간의 이전 Q&A 세션

이 다양한 데이터는 에이전트가 다음을 가능하게 합니다:
- 사용자의 기술적 맥락을 이해
- 추천 사항에 모범 사례 적용
- 이전 성공적인 상호작용에서 학습


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 데이터를 지식 그래프로 변환하기

이제 원시 텍스트를 구조화된 메모리로 변환합니다. 이 과정은 다음과 같습니다:

1. **NodeSets에 데이터를 추가**: 정보를 논리적인 카테고리로 정리
   - `developer_data`: 개발자 프로필 및 대화
   - `principles_data`: Python 모범 사례 및 가이드라인

2. **Cognify Pipeline 실행**: 엔티티와 관계를 추출하고 임베딩 생성
   - 주요 개념 식별
   - 관련 정보 간의 의미론적 연결 생성
   - 벡터 임베딩 생성

LLM이 텍스트를 처리하고 그래프 구조를 구축하는 동안 약간의 시간이 걸릴 수 있습니다:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 지식 그래프 시각화

우리의 지식 그래프 구조를 살펴봅시다. 시각화는 다음을 보여줍니다:
- **노드**: 텍스트에서 추출된 엔티티(개념, 기술, 사람)
- **엣지**: 엔티티 간의 관계와 연결
- **클러스터**: 의미적 유사성에 따라 그룹화된 관련 개념들

생성된 HTML 파일을 브라우저에서 열어 그래프를 상호작용하며 탐색하세요:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Memify로 기억 강화하기

`memify()` 함수는 지식 그래프를 분석하고 데이터에 대한 지능적인 규칙을 생성합니다. 이 과정은 다음을 포함합니다:
- 패턴과 모범 사례 식별
- 콘텐츠를 기반으로 실행 가능한 지침 생성
- 다양한 지식 영역 간의 관계 설정

이러한 규칙은 에이전트가 질문에 답할 때 더 나은 결정을 내릴 수 있도록 돕습니다. 두 번째 시각화를 캡처하면 그래프가 풍부해진 후 어떻게 밀도가 높아지는지 비교할 수 있습니다.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 2부: 지능형 메모리 검색

### 데모 1: 문서 간 지식 통합

지식 그래프가 구축되었으니, Cognee가 여러 출처의 정보를 결합하여 복잡한 질문에 어떻게 답변하는지 테스트해봅시다.

첫 번째 쿼리는 다음을 보여줍니다:
- **의미 이해**: 명시적으로 언급되지 않은 경우에도 관련 개념 찾기
- **교차 참조**: 개발자 프로필과 Python 원칙 결합
- **맥락적 추론**: 특정 구현에 모범 사례 적용

### 데모 2: NodeSets를 활용한 필터링 검색

두 번째 쿼리는 지식 그래프의 특정 하위 집합을 대상으로 하는 방법을 보여줍니다:
- `node_name` 매개변수를 사용하여 `principles_data` 내에서만 검색
- 특정 지식 도메인에서 집중된 답변 제공
- 도메인별 정보가 필요할 때 유용


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 3부: 세션 관리 설정

### 대화 메모리 활성화

세션 관리는 여러 상호작용 간에 문맥을 유지하는 데 매우 중요합니다. 여기에서는 다음을 수행합니다:

1. **사용자 컨텍스트 초기화**: 세션 추적을 위해 사용자 프로필을 생성하거나 가져오기
2. **캐시 엔진 구성**: 대화 기록을 저장하기 위해 Redis에 연결
3. **세션 변수 활성화**: 쿼리 간에 지속되는 컨텍스트 변수를 설정

> ⚠️ **중요**: Redis가 실행 중이어야 하며 환경에서 `CACHING=true`로 설정되어 있어야 합니다


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ 도우미 함수: 세션 기록 보기

이 유틸리티 함수는 Redis에 저장된 대화 기록을 확인할 수 있게 해줍니다. 이는 다음과 같은 경우에 유용합니다:
- 세션 관리 디버깅
- 대화가 캐시되고 있는지 확인
- 에이전트가 사용할 수 있는 컨텍스트 이해


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## 세션 1: 비동기 지원 실습 — 첫 번째 질문

`async-support-lab` 세션을 시작하며 대규모 웹 스크래퍼를 위한 텔레메트리 친화적인 asyncio 패턴에 대해 질문하세요. 그래프는 이미 asyncio, aiohttp, 모니터링 실습에 대해 알고 있으므로, 응답은 이전 대화를 반영하면서도 새로운 질문에 맞춘 답변이어야 합니다.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## 첫 번째 교환 후 세션 1 메모리 확인

초기 질문 직후 `show_history(session_1)`을 실행하면 Cognee가 프롬프트와 응답을 모두 Redis에 기록했음을 확인할 수 있습니다. 동시성 가이던스가 포함된 하나의 항목을 볼 수 있을 것입니다.


In [ ]:
await show_history(session_1)

## 세션 1: 데이터 모델에 대한 후속 논의

다음으로, "dataclasses와 Pydantic 중 언제 선택해야 할까?"라는 질문을 같은 세션 ID를 사용하여 탐구합니다. Cognee는 Python 원칙과 이전 FastAPI 대화를 결합하여 세부적인 조언을 제공해야 하며, 명명된 세션 내에서 문맥이 이어진다는 것을 보여줍니다.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## 세션 1 기록에 두 개의 턴이 포함되어 있는지 확인하기

다시 `show_history(session_1)`를 호출하면 두 개의 Q&A 항목이 표시되어야 합니다. 이는 Mem0 실험실의 "기억 재생" 단계와 일치하며, 추가적인 턴이 동일한 대화 기록을 확장한다는 것을 증명합니다.


In [ ]:
await show_history(session_1)

## 세션 2: 디자인 리뷰 스레드 — 새로운 세션

스레드 간의 분리를 보여주기 위해 `design-review-session`을 시작하고 사고 리뷰를 위한 로깅 가이드를 요청합니다. 기본 지식 베이스는 동일하지만, 새로운 세션 ID는 대화 기록을 분리합니다.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## 검토 세션 2 기록

`show_history(session_2)`는 디자인 검토 프롬프트/응답 쌍만 나열해야 합니다. 세션 1과 비교하여 Cognee가 공유 지식 그래프를 재사용하면서도 독립적인 기록을 유지하는 방식을 강조하세요.


In [ ]:
await show_history(session_2)

## 요약

축하합니다! 이제 코딩 어시스턴트에 Cognee로 구동되는 진정한 장기 기억 레이어를 추가했습니다.

이 튜토리얼에서는 개발자 콘텐츠(코드, 문서, 채팅)를 그래프와 벡터 메모리로 변환하여 에이전트가 검색하고, 추론하며, 지속적으로 개선할 수 있도록 만드는 방법을 배웠습니다.

배운 내용

1. **원시 텍스트에서 AI 메모리로**: Cognee가 비구조화 데이터를 수집하고 벡터와 지식 그래프 아키텍처를 결합하여 지능적이고 검색 가능한 메모리로 변환하는 방법.

2. **memify를 활용한 그래프 강화**: 기본적인 그래프 생성 단계를 넘어 memify를 사용하여 기존 그래프에 파생된 사실과 더 풍부한 관계를 추가하는 방법.

3. **다양한 검색 전략**: 에이전트의 필요에 따라 메모리를 다양한 검색 유형(그래프 인식 Q&A, RAG 스타일 완성, 인사이트, 원시 텍스트 조각, 코드 검색 등)으로 쿼리하는 방법.

4. **시각적 탐색**: 그래프 시각화와 Cognee UI를 사용하여 Cognee가 구축한 내용을 검사하고 디버그하며 지식이 어떻게 구조화되어 있는지 실제로 확인하는 방법.

5. **세션 인식 메모리**: 세션별 컨텍스트와 지속적인 의미 메모리를 결합하여 에이전트가 사용자 간 정보 누출 없이 실행 간 기억을 유지할 수 있도록 하는 방법.


## 주요 요점
1. 임베딩을 기반으로 한 지식 그래프로서의 메모리

   - **구조화된 이해**: Cognee는 벡터 저장소와 그래프 저장소를 결합하여 데이터를 의미에 따라 검색 가능하게 하고 관계로 연결합니다. Cognee는 기본적으로 파일 기반 데이터베이스를 사용합니다 (벡터 데이터베이스는 LanceDB, 그래프 데이터베이스는 Kuzu).

   - **관계 인식 검색**: 답변은 "유사한 텍스트"뿐만 아니라 엔티티 간의 관계를 기반으로 할 수 있습니다.

   - **진화하는 메모리**: 메모리 계층은 하나의 연결된 그래프로 진화하고 성장하며 계속 쿼리 가능합니다.

2. 검색 및 추론 모드
   - **하이브리드 검색**: 검색은 벡터 유사성, 그래프 구조, LLM 추론을 결합하여 원시 청크 조회에서 그래프 인식 질문 응답까지 제공합니다.

   - **작업에 맞는 모드 선택**: 자연어 답변이 필요한 경우 완성형 모드를 사용하고, 에이전트가 원시 컨텍스트를 필요로 하거나 자체 추론을 주도해야 할 때는 청크/요약/그래프 모드를 사용하세요.

3. 개인화된 세션 인식 에이전트
   - **세션 컨텍스트 + 장기 메모리**: Cognee는 단기 "스레드" 컨텍스트를 장기적인 사용자 또는 조직 수준의 메모리와 분리하여 관리합니다.

## 실제 응용 사례

1. **특정 분야 AI 에이전트**

   이 노트북의 패턴을 사용하여 Cognee를 검색 및 추론 핵심으로 활용하는 도메인 지능형 코파일럿을 구축하세요:

   - **개발자 코파일럿**: 코드 리뷰, 사고 분석, 아키텍처 지원 도구로 코드, API, 설계 문서, 티켓을 단일 메모리 그래프로 탐색합니다.

   - **고객 대면 코파일럿**: 제품 문서, FAQ, CRM 노트, 과거 티켓에서 그래프 인식 검색과 인용된 답변을 제공하는 지원 또는 성공 에이전트.

   - **내부 전문가 코파일럿**: 고립된 PDF 대신 상호 연결된 규칙, 지침, 과거 결정을 기반으로 추론하는 정책, 법률, 보안 지원 도구.

   Cognee는 AI 에이전트를 위한 지속적이고 정확한 메모리로 명확히 자리 잡고 있으며, 에이전트 뒤에서 작동하며 벡터 저장소와 맞춤형 그래프 코드를 대체하는 살아있는 지식 그래프를 제공합니다.

2. **데이터 사일로를 하나의 메모리로 통합**

   동일한 접근 방식은 분산된 소스를 통합 메모리 계층으로 구축하는 데도 도움이 됩니다:

   - **사일로에서 하나의 그래프로**: 구조화된 데이터(예: 데이터베이스)와 비구조화된 데이터(예: 문서, 채팅)를 별도의 인덱스 대신 임베딩으로 지원되는 단일 그래프로 수집합니다.

   - **인용과 함께 교차 소스 추론**: 그래프를 통해 로그, 메트릭, 문서를 "조인"하여 모든 것을 다단계로 추론하고, 여전히 출처가 명확한 답변을 반환합니다.

   - **지식 허브**: 은행업이나 교육과 같은 도메인에서는 Cognee를 사용하여 PDF, 내부 시스템, 앱 데이터를 하나의 지식 그래프로 통합하고 벡터를 활용하여 에이전트가 정확하고 인용된 컨텍스트로 질문에 답할 수 있도록 합니다.

## 다음 단계

핵심 메모리 루프를 구현했습니다. 다음은 스스로 시도해볼 수 있는 자연스러운 확장입니다 ([Cognee 문서](https://docs.cognee.ai/)에서 자세한 내용을 확인하세요):

1. **시간 인식 실험**: 텍스트에서 이벤트와 타임스탬프를 추출하기 위해 시간 인식을 활성화하세요.

2. **온톨로지 기반 추론 도입**: 도메인에 대한 OWL 온톨로지를 정의하세요. Cognee의 온톨로지 지원을 사용하여 추출된 엔티티와 관계를 해당 스키마에 기반하도록 하여 그래프 품질과 도메인별 답변을 개선하세요.

3. **피드백 루프 추가**: 실제 사용자 피드백을 통해 Cognee가 그래프 엣지 가중치를 조정하도록 하여 검색이 정적 상태에 머무르지 않고 시간이 지남에 따라 개선되도록 하세요.

4. **개인화 및 세션 동작 조정**: 사용자 ID, 테넌트, 데이터 세트를 사용하여 공유 메모리 엔진에 대해 각 개인이나 팀이 고유한 뷰를 가질 수 있도록 하세요.

5. **더 복잡한 에이전트로 확장**: Cognee를 에이전트 프레임워크에 연결하여 동일한 메모리 계층을 공유하는 다중 에이전트 시스템을 구축하세요. *Microsoft Agent Framework x Cognee 플러그인은 곧 출시됩니다.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 출처로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임지지 않습니다.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
